In [1]:
import xml.etree.ElementTree as ET

In [2]:
def extract_text_by_page(xml_text, page_number):
    """指定したページのテキストを取得"""
    root = ET.fromstring(xml_text)
    page = root.find(".//page[@n='{}']".format(page_number))
    if page is not None:
        return "".join(page.itertext()).strip()
    return "該当ページが見つかりません"

In [3]:
def extract_recursive_text(section):
    """指定された<section>の下層を再帰的に取得"""
    text_list = []
    
    if section.text and section.text.strip():
        text_list.append(section.text.strip())
    
    for child in section.findall(".//section"):
        text_list.extend(extract_recursive_text(child))
    
    return text_list

In [4]:
def extract_sections_by_type(xml_text, section_type):
    """指定したtypeのセクションのテキストを抽出（再帰的）"""
    root = ET.fromstring(xml_text)
    extracted_text = []

    for section in root.findall(".//section[@type='{}']".format(section_type)):
        extracted_text.extend(extract_recursive_text(section))
    
    return "\n\n".join(extracted_text)

In [5]:
def extract_text_by_page_and_type(xml_text, page_list, section_type):
    """
    指定したページとtypeのセクションのテキストを抽出（再帰的）
    :param xml_text: XML形式の文字列
    :param page_list: 取得したいページ番号のリスト
    :param section_type: 抽出したいセクションのtype属性値
    :return: 該当するセクションのテキスト
    """
    root = ET.fromstring(xml_text)
    extracted_text = []
    
    for page_num in page_list:
        page = root.find(f".//page[@n='{page_num}']")
        if page is not None:
            for section in page.findall(f".//section[@type='{section_type}']"):
                extracted_text.extend(extract_recursive_text(section))
    
    return "\n\n".join(extracted_text)

In [6]:
# 使用例
xml_data = """<Documents>
    <page n='1'>
        <section type='1'>
            <section type='a'>
                section 1-aのテキスト
                <section type='x'>
                    section 1-a-xのテキスト
                </section>
            </section>
            <section type='b'>
                section 1-bのテキスト（前半）
            </section>
        </section>
    </page>
    <page n='2'>
        <section type='1'>
            <section type='b'>
                section 1-bのテキスト（後半）
            </section>
        </section>
        <section type='2'>
            <section type='a'>
                section 2-aのテキスト
                <section type='x'>
                    section 2-a-xのテキスト
                </section>
            </section>
            <section type='b'>
                section 2-bのテキスト
            </section>
        </section>
    </page>
</Documents>"""

# "1" を指定 → その配下の全ての <section> のテキストを再帰的に取得
print("\n=== page='1' を指定 ===")
print(extract_text_by_page(xml_data, '1'))

print("=== type='1' を指定 ===")
print(extract_sections_by_type(xml_data, "1"))

print("\n=== type='b' を指定 ===")
print(extract_sections_by_type(xml_data, "b"))

# 指定ページとtypeの組み合わせで取得
page_list = [1, 2]
section_type = '1'
result = extract_text_by_page_and_type(xml_data, page_list, section_type)

print("=== 指定されたページとtypeの結果 ===")
print(result)

page_list = [1, 2]
section_type = 'b'
result = extract_text_by_page_and_type(xml_data, page_list, section_type)

print("=== 指定されたページとtypeの結果 ===")
print(result)



=== page='1' を指定 ===
section 1-aのテキスト
                
                    section 1-a-xのテキスト
                
            
            
                section 1-bのテキスト（前半）
=== type='1' を指定 ===
section 1-aのテキスト

section 1-a-xのテキスト

section 1-a-xのテキスト

section 1-bのテキスト（前半）

section 1-bのテキスト（後半）

=== type='b' を指定 ===
section 1-bのテキスト（前半）

section 1-bのテキスト（後半）

section 2-bのテキスト
=== 指定されたページとtypeの結果 ===
section 1-aのテキスト

section 1-a-xのテキスト

section 1-a-xのテキスト

section 1-bのテキスト（前半）

section 1-bのテキスト（後半）
=== 指定されたページとtypeの結果 ===
section 1-bのテキスト（前半）

section 1-bのテキスト（後半）

section 2-bのテキスト
